# Attentional Networks in Computer Vision
Prepared by comp411 Teaching Unit (TA Can Küçüksözen) in the context of Computer Vision with Deep Learning Course. Do not hesitate to ask in case you have any questions, contact me at: ckucuksozen19@ku.edu.tr

Up until this point, we have worked with deep fully-connected networks, convolutional networks and recurrent networks using them to explore different optimization strategies and network architectures. Fully-connected networks are a good testbed for experimentation because they are very computationally efficient, on the other hand, most successful image processing methods use convolutional networks. However recent state-of-the-art results on computer vision realm are acquired using Attentional layers and Transformer architectures.

First you will implement several layer types that are used in fully attentional networks. You will then use these layers to train an Attentional Image Classification network, specifically a smaller version of Vision Transformer (VIT) on the CIFAR-10 dataset. The original paper can be accessed via the following link: https://arxiv.org/pdf/2010.11929.pdf

# Part I. Preparation

First, we load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In previous parts of the assignment we had to write our own code to download the CIFAR-10 dataset, preprocess it, and iterate through it in minibatches; PyTorch provides convenient tools to automate this process for us.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [2]:
PYTORCH_ENABLE_MPS_FALLBACK=1 

In [3]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./comp411/datasets', train=True, download=False,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./comp411/datasets', train=True, download=False,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./comp411/datasets', train=False, download=False, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

You have an option to **use GPU by setting the flag to True below**. It is not necessary to use GPU for this assignment. Note that if your computer does not have CUDA enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.

The global variables `dtype` and `device` will control the data types throughout this assignment. 

In [4]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


# Part II. Barebones Transformers: Self-Attentional Layer

Here you will complete the implementation of the Pytorch nn.module `SelfAttention`, which will perform the forward pass of a self-attentional layer. Our implementation of the SelfAttentional layer will include three distinct fully connected layers which will be responsible of:

1. A fully connected layer, `W_Q`, which will be used to project our input into `queries`
2. A fully connected layer, `W_K`, which will be used to project our input into `keys`
3. A fully connected layer, `W_V`, which will be used to project our input into `values`

After defining such three fully connected layers, and obtain our `queries, keys, and values` variables at the beginning of our forward pass, the following operations should be carried out in order to complete the attentional layer implementation.

1. Seperate each of `query, key, and value` projections into their respective heads. In other words, split the feature vector dimension of each matrix into necessarry number of chunks.

2. Compute the `Attention Scores` between each pair of sequence elements via conducting a scaled dot product operation between every pair of `queries` and `keys`. Note that `Attention Scores` matrix should have the size of `[# of queries , # of keys]`

3. Calculate the `Attention Weights` of each query by applying the non-linear `Softmax` normalization accross the `keys` dimension of the `Attention Scores` matrix.

4. Obtain the output combination of `values` by matrix multiplying `Attention Weights` with `values`

5. Reassemble heads into one flat vector and return the output.

**HINT**: For a more detailed explanation of the self attentional layer, examine the Appendix A of the original ViT manuscript here:  https://arxiv.org/pdf/2010.11929.pdf 

In [5]:
class SelfAttention(nn.Module):
    
    def __init__(self, input_dims, head_dims=128, num_heads=2,  bias=False):
        super(SelfAttention, self).__init__()
        
        ## initialize module's instance variables
        self.input_dims = input_dims
        self.head_dims = head_dims
        self.num_heads = num_heads
        self.proj_dims = head_dims * num_heads
        
        ## Declare module's parameters
        self.W_Q = nn.Linear(input_dims, self.proj_dims,bias=bias)
        self.W_K = nn.Linear(input_dims, self.proj_dims,bias=bias)
        self.W_V = nn.Linear(input_dims, self.proj_dims,bias=bias)

        self.init_weights()
        
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.fill_(0.1)

    def forward(self, x):
        ## Input of shape, [B, N, D] where:
        ## - B denotes the batch size
        ## - N denotes number of sequence elements. I.e. the number of patches + the class token 
        ## - D corresponds to model dimensionality
        b,n,d = x.shape
        
        ## Construct queries,keys,values
        q_ = self.W_Q(x)
        k_ = self.W_K(x)
        v_ = self.W_V(x)
        
        ## Seperate q,k,v into their corresponding heads,
        ## After this operation each q,k,v will have the shape: [B,H,N,D//H] where
        ## - B denotes the batch size
        ## - H denotes number of heads
        ## - N denotes number of sequence elements. I.e. the number of patches + the class token 
        ## - D//H corresponds to per head dimensionality
        q, k, v = map(lambda z: torch.reshape(z, (b,n,self.num_heads,self.head_dims)).permute(0,2,1,3), [q_,k_,v_])
       
        #########################################################################################
        # TODO: Complete the forward pass of the SelfAttention layer, follow the comments below #
        #########################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        
        ## Compute attention logits. Note that this operation is conducted as a
        ## batched matrix multiplication between q and k, the output is scaled by 1/(D//H)^(1/2)
        ## inputs are queries and keys that are both of size [B,H,N,D//H]
        ## Output Attention logits should have the size: [B,H,N,N]
        
        attention_logits =  torch.matmul(q, k.transpose(-2, -1)) * (1 / ((d//self.num_heads)**(1/2)))
    
        ## Compute attention Weights. Note that this operation is conducted as a
        ## Softmax Normalization across the keys dimension. 
        ## Hint: You can apply the Softmax operation across the final dimension

        attention_weights = torch.softmax(attention_logits, dim = -1)
        
        ## Compute output values. Note that this operation is conducted as a 
        ## batched matrix multiplication between the Attention Weights matrix and 
        ## the values tensor. After computing output values, the output should be reshaped
        ## Inputs are Attention Weights with size [B, H, N, N], values with size [B, H, N, D//H]
        ## Output should be of size [B, N, D]
        ## Hint: you should use torch.matmul, torch.permute, torch.reshape in that order
        
        attn_out = torch.matmul(attention_weights, v).permute(0, 2, 1, 3).reshape(-1, n, self.proj_dims)
        
        
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             
        ################################################################################
    
        return attn_out

After defining the forward pass of the Self-Attentional Layer above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 64).

In [6]:
def test_self_attn_layer():
    x = torch.zeros((64, 16, 32), dtype=dtype)  # minibatch size 64, sequence elements size 16, feature channels size 32
    layer = SelfAttention(32,64,4)
    out = layer(x)
    print(out.size())  # you should see [64,16,256]
test_self_attn_layer()

torch.Size([64, 16, 256])


# Part III. Barebones Transformers: Transformer Encoder Block

Here you will complete the implementation of the Pytorch nn.module `TransformerBlock`, which will perform the forward pass of a Transfomer Encoder Block. You can refer to Figure 1 of the original manuscript of ViT from this link: https://arxiv.org/pdf/2010.11929.pdf in order to get yourself familiar with the architecture.



In [7]:
## Implementation of a two layer GELU activated Fully Connected Network is provided for you below:

class MLP(nn.Module):
    def __init__(self, input_dims, hidden_dims, output_dims, bias=True):
        super().__init__()
        
        self.fc_1 = nn.Linear(input_dims, hidden_dims, bias=bias)
        self.fc_2 = nn.Linear(hidden_dims, output_dims, bias=bias)
        
        self.init_weights()
        
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                if m.bias is not None:
                    m.bias.data.fill_(0.1)
        
    def forward(self, x):
        o = F.relu(self.fc_1(x))
        o = self.fc_2(o)
        return o

In [8]:
## Build from scratch a TransformerBlock Module. Note that the architecture of this
## module follows a simple computational pipeline:
## input --> layernorm --> SelfAttention --> skip connection 
##       --> layernorm --> MLP ---> skip connection ---> output
## Note that the TransformerBlock module works on a single hidden dimension hidden_dims,
## in order to faciliate skip connections with ease. Be careful about the input arguments
## to the SelfAttention block.


class TransformerBlock(nn.Module):
    def __init__(self, hidden_dims, num_heads=4, bias=False):
        super(TransformerBlock, self).__init__()
        
###############################################################
# TODO: Complete the consturctor of  TransformerBlock module  #
###############################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)****
        self.norm_layer = nn.LayerNorm(hidden_dims)
        self.attention = SelfAttention(hidden_dims, hidden_dims//num_heads, num_heads, bias=bias)
        self.norm_layer2 = nn.LayerNorm(hidden_dims)
        self.mlp= MLP(hidden_dims, hidden_dims, hidden_dims, bias=bias)
        
        
# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
###################################################################
#                                 END OF YOUR CODE                #             
###################################################################
        
    def forward(self, x):
        
##############################################################
# TODO: Complete the forward of TransformerBlock module      #
##############################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)****

        norm_layer = self.norm_layer(x)
        attention  = self.attention(norm_layer)
        norm_layer2       = self.norm_layer2(attention + x)
        mlp                  = self.mlp(norm_layer2)
        output               = mlp + attention + x
        return output
        
 # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
###################################################################
#                                 END OF YOUR CODE                #             
###################################################################

After defining the forward pass of the Transformer Block Layer above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 16, 128).

In [9]:
def test_transfomerblock_layer():
    x = torch.zeros((64, 16, 128), dtype=dtype)  # minibatch size 64, sequence elements size 16, feature channels size 128
    layer = TransformerBlock(128,4) # hidden dims size 128, heads size 4
    out = layer(x)
    print(out.size()) 
test_transfomerblock_layer()

torch.Size([64, 16, 128])


# Part IV The Vision Transformer (ViT)

The final implementation for the Pytorch nn.module `ViT` is given to you below, which will perform the forward pass of the Vision Transformer. Study it and get yourself familiar with the API.


In [10]:
class ViT(nn.Module):
    def __init__(self, hidden_dims, input_dims=3, output_dims=10, num_trans_layers = 4, num_heads=4, image_k=32, patch_k=4, bias=False):
        super(ViT, self).__init__()
                
        ## initialize module's instance variables
        self.hidden_dims = hidden_dims
        self.input_dims = input_dims
        self.output_dims = output_dims
        self.num_trans_layers = num_trans_layers
        self.num_heads = num_heads
        self.image_k = image_k
        self.patch_k = patch_k
        
        self.image_height = self.image_width = image_k
        self.patch_height = self.patch_width = patch_k
        
        assert self.image_height % self.patch_height == 0 and self.image_width % self.patch_width == 0,\
                'Image size must be divisible by the patch size.'

        self.num_patches = (self.image_height // self.patch_height) * (self.image_width // self.patch_width)
        self.patch_flat_len = self.patch_height * self.patch_width
        
        ## Declare module's parameters
        
        ## ViT's flattened patch embedding projection:
        self.linear_embed = nn.Linear(self.input_dims*self.patch_flat_len, self.hidden_dims)
        
        ## Learnable positional embeddings, an embedding is learned for each patch location and the class token
        self.pos_embedding = nn.Parameter(torch.randn(1, self.num_patches + 1, self.hidden_dims))
        
        ## Learnable classt token and its index among attention sequence elements.
        self.cls_token = nn.Parameter(torch.randn(1,1,self.hidden_dims))
        self.cls_index = torch.LongTensor([0])
        
        ## Declare cascaded Transformer blocks:
        transformer_encoder_list = []
        for _ in range(self.num_trans_layers):
            transformer_encoder_list.append(TransformerBlock(self.hidden_dims, self.num_heads, bias))
        self.transformer_encoder = nn.Sequential(*transformer_encoder_list)
        
        ## Declare the output mlp:
        self.out_mlp = MLP(self.hidden_dims, self.hidden_dims, self.output_dims)
         
    def unfold(self, x, f = 7, st = 4, p = 0):
        ## Create sliding window pathes using nn.Functional.unfold
        ## Input dimensions: [B,D,H,W] where
        ## --B : input batch size
        ## --D : input channels
        ## --H, W: input height and width
        ## Output dimensions: [B,N,H*W,D]
        ## --N : number of patches, decided according to sliding window kernel size (f),
        ##      sliding window stride and padding.
        b,d,h,w = x.shape
        x_unf = F.unfold(x, (f,f), stride=st, padding=p)    
        x_unf = torch.reshape(x_unf.permute(0,2,1), (b,-1,d,f*f)).transpose(-1,-2)
        n = x_unf.size(1)
        return x_unf,n
    
    def forward(self, x):
        b = x.size(0)
        ## create sliding window patches from the input image
        x_patches,n = self.unfold(x, self.patch_height, self.patch_height, 0)
        ## flatten each patch into a 1d vector: i.e. 3x4x4 image patch turned into 1x1x48
        x_patch_flat = torch.reshape(x_patches, (b,n,-1))
        ## linearly embed each flattened patch
        x_embed = self.linear_embed(x_patch_flat)
        
        ## retrieve class token 
        cls_tokens = self.cls_token.repeat(b,1,1)
        ## concatanate class token to input patches
        xcls_embed = torch.cat([cls_tokens, x_embed], dim=-2)
        
        ## add positional embedding to input patches + class token 
        xcls_pos_embed = xcls_embed + self.pos_embedding
        
        ## pass through the transformer encoder
        trans_out = self.transformer_encoder(xcls_pos_embed)
        
        ## select the class token 
        out_cls_token = torch.index_select(trans_out, -2, self.cls_index.to(trans_out.device))
        
        ## create output
        out = self.out_mlp(out_cls_token)
        
        return out.squeeze(-2)

After defining the forward pass of the ViT above, run the following cell to test your implementation.

When you run this function, output should have shape (64, 10).

In [11]:
def test_vit():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size 3,32,32
    model = ViT(hidden_dims=128, input_dims=3, output_dims=10, num_trans_layers = 4, num_heads=4, image_k=32, patch_k=4)
    out = model(x)
    print(out.size()) 
test_vit()

torch.Size([64, 10])


# Part V. Train the ViT

### Check Accuracy
Given any minibatch of input data and desired targets, we can check the classification accuracy of a neural network. 

The check_batch_accuracy function is provided for you below:

In [12]:
def check_batch_accuracy(out, target,eps=1e-7):
    b, c = out.shape
    with torch.no_grad():
        _, pred = out.max(-1) 
        correct = np.sum(np.equal(pred.cpu().numpy(), target.cpu().numpy()))
    return correct, float(correct) / (b)

### Training Loop
As we have already seen in the Second Assignment, in our PyTorch based training loops, we use an Optimizer object from the `torch.optim` package, which abstract the notion of an optimization algorithm and provides implementations of most of the algorithms commonly used to optimize neural networks.

In [13]:
def train(network, optimizer, trainloader):
    """
    Train a model on CIFAR-10 using the PyTorch Module API for a single epoch
    
    Inputs:
    - network: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - trainloader: Iterable DataLoader object that fetches the minibatches
    
    Returns: overall training accuracy for the epoch
    """
    print('\nEpoch: %d' % epoch)
    network.train()  # put model to training mode
    network = network.to(device=device)  # move the model parameters to CPU/GPU
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = Variable(inputs.to(device)), targets.to(device)  # move to device, e.g. GPU
            
        outputs = network(inputs)
        loss =  F.cross_entropy(outputs, targets)
            
        # Zero out all of the gradients for the variables which the optimizer
        # will update.
        optimizer.zero_grad() 

        # This is the backwards pass: compute the gradient of the loss with
        # respect to each  parameter of the model.
        loss.backward()
            
        # Actually update the parameters of the model using the gradients
        # computed by the backwards pass.
        optimizer.step()
            
        loss = loss.detach()
        train_loss += loss.item()
        correct_p, _ = check_batch_accuracy(outputs, targets) 
        correct += correct_p
        total += targets.size(0)

        print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
        % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
    return 100.*correct/total

### Evaluation Loop
We have also prepared a Evaluation loop in order to determine our networks capabilities in terms of classification accuracy on a given dataset, either the training, or the validation split

In [14]:
def evaluate(network, evalloader):
    """
    Evaluate a model on CIFAR-10 using the PyTorch Module API for a single epoch
    
    Inputs:
    - network: A PyTorch Module giving the model to train.
    - evalloader: Iterable DataLoader object that fetches the minibatches
    
    Returns: overall evaluation accuracy for the epoch
    """
    network.eval() # put model to evaluation mode
    network = network.to(device=device)  # move the model parameters to CPU/GPU
    eval_loss = 0
    correct = 0
    total = 0
    print('\n---- Evaluation in process ----')
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(evalloader):
            inputs, targets = inputs.to(device), targets.to(device) # move to device, e.g. GPU
            outputs = network(inputs)
            loss = F.cross_entropy(outputs, targets)
            
            eval_loss += loss.item()
            correct_p, _ = check_batch_accuracy(outputs, targets)
            correct += correct_p
            total += targets.size(0)
            print('Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (eval_loss/(batch_idx+1), 100.*correct/total, correct, total))
    return 100.*correct/total

### Overfit a ViT
Now we are ready to run the training loop. A nice trick is to train your model with just a few training samples in order to see if your implementation is actually bug free. 

Simply pass the input size, hidden layer size, and number of classes (i.e. output size) to the constructor of `ViT`. 

You also need to define an optimizer that tracks all the learnable parameters inside `ViT`. We prefer to use `Adam` optimizer for this part.

You should be able to overfit small datasets, which will result in very high training accuracy and comparatively low validation accuracy.

In [15]:
sample_idx_tr = torch.randperm(len(cifar10_train))[:100]
sample_idx_val = torch.randperm(len(cifar10_train))[-100:]

trainset_sub = torch.utils.data.Subset(cifar10_train, sample_idx_tr)
valset_sub = torch.utils.data.Subset(cifar10_train, sample_idx_val)

print("For overfitting experiments, the subset of the dataset that is used has {} sample images".format(len(trainset_sub)))

batch_size_sub = 25
trainloader_sub = torch.utils.data.DataLoader(trainset_sub, batch_size=batch_size_sub, shuffle=True)
valloader_sub = torch.utils.data.DataLoader(valset_sub, batch_size=batch_size_sub, shuffle=False)

print('==> Data ready, batchsize = {}'.format(batch_size_sub))

For overfitting experiments, the subset of the dataset that is used has 100 sample images
==> Data ready, batchsize = 25


In [17]:
learning_rate = 0.002
regularization_val = 1e-6
input_dims = 3
hidden_dims = 128
output_dims=10
num_trans_layers = 4
num_heads=4
image_k=32
patch_k=4

model = None
optimizer = None

################################################################################
# TODO: Instantiate your ViT model and a corresponding optimizer #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

network = ViT(hidden_dims, 
              input_dims, 
              output_dims, 
              num_trans_layers, 
              num_heads, 
              image_k, 
              patch_k)

optimizer = torch.optim.Adam(network.parameters(), lr = learning_rate)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

tr_accs=[]
eval_accs=[]
for epoch in range(15):
    tr_acc = train(network, optimizer, trainloader_sub)
    print('Epoch {} of training is completed, Training accuracy for this epoch is {}'\
              .format(epoch, tr_acc))  
    
    eval_acc = evaluate(network, valloader_sub)
    print('Evaluation of Epoch {} is completed, Validation accuracy for this epoch is {}'\
              .format(epoch, eval_acc))  
    tr_accs.append(tr_acc)
    eval_accs.append(eval_acc)
    
print("\nFinal train set accuracy is {}".format(tr_accs[-1]))
print("Final val set accuracy is {}".format(eval_accs[-1]))


Epoch: 0
Loss: 4.320 | Acc: 8.000% (2/25)
Loss: 3.928 | Acc: 8.000% (4/50)
Loss: 4.095 | Acc: 6.667% (5/75)
Loss: 4.393 | Acc: 7.000% (7/100)
Epoch 0 of training is completed, Training accuracy for this epoch is 7.0

---- Evaluation in process ----
Loss: 2.856 | Acc: 20.000% (5/25)
Loss: 3.321 | Acc: 16.000% (8/50)
Loss: 3.180 | Acc: 16.000% (12/75)
Loss: 3.317 | Acc: 15.000% (15/100)
Evaluation of Epoch 0 is completed, Validation accuracy for this epoch is 15.0

Epoch: 1
Loss: 4.164 | Acc: 12.000% (3/25)
Loss: 3.692 | Acc: 16.000% (8/50)
Loss: 3.571 | Acc: 14.667% (11/75)
Loss: 3.351 | Acc: 14.000% (14/100)
Epoch 1 of training is completed, Training accuracy for this epoch is 14.0

---- Evaluation in process ----
Loss: 3.179 | Acc: 0.000% (0/25)
Loss: 3.161 | Acc: 8.000% (4/50)
Loss: 3.061 | Acc: 10.667% (8/75)
Loss: 3.016 | Acc: 11.000% (11/100)
Evaluation of Epoch 1 is completed, Validation accuracy for this epoch is 11.0

Epoch: 2
Loss: 2.103 | Acc: 20.000% (5/25)
Loss: 2.362 | Ac

## Train the net
By training the four-layer ViT network for three epochs, with untuned hyperparameters that are initialized as below,  you should achieve greater than 50% accuracy both on the training set and the test set:

In [19]:
learning_rate = 0.002
regularization_val = 1e-6
input_dims = 3
hidden_dims = 128
output_dims=10
num_trans_layers = 4
num_heads=4
image_k=32
patch_k=4

network = None
optimizer = None

################################################################################
# TODO: Instantiate your ViT model and a corresponding optimizer #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

network = ViT(hidden_dims, 
            input_dims, 
            output_dims, 
            num_trans_layers, 
            num_heads, 
            image_k, 
            patch_k)

optimizer = torch.optim.Adam(network.parameters(), lr = learning_rate)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

tr_accs=[]
test_accs=[]
for epoch in range(3):
    tr_acc = train(network, optimizer, loader_train)
    print('Epoch {} of training is completed, Training accuracy for this epoch is {}'\
              .format(epoch, tr_acc))  
    
    test_acc = evaluate(network, loader_test)
    print('Evaluation of Epoch {} is completed, Test accuracy for this epoch is {}'\
              .format(epoch, test_acc))  
    
    tr_accs.append(tr_acc)
    test_accs.append(test_acc)
    
print("\nFinal train set accuracy is {}".format(tr_accs[-1]))
print("Final test set accuracy is {}".format(test_accs[-1]))


Epoch: 0
Loss: 2.748 | Acc: 15.625% (10/64)
Loss: 3.407 | Acc: 16.406% (21/128)
Loss: 3.894 | Acc: 15.104% (29/192)
Loss: 4.345 | Acc: 14.453% (37/256)
Loss: 4.359 | Acc: 15.312% (49/320)
Loss: 4.221 | Acc: 15.625% (60/384)
Loss: 4.091 | Acc: 15.625% (70/448)
Loss: 3.982 | Acc: 15.430% (79/512)
Loss: 3.905 | Acc: 14.931% (86/576)
Loss: 3.798 | Acc: 15.000% (96/640)
Loss: 3.690 | Acc: 15.199% (107/704)
Loss: 3.579 | Acc: 15.885% (122/768)
Loss: 3.550 | Acc: 15.865% (132/832)
Loss: 3.509 | Acc: 16.071% (144/896)
Loss: 3.466 | Acc: 15.833% (152/960)
Loss: 3.400 | Acc: 16.309% (167/1024)
Loss: 3.341 | Acc: 16.452% (179/1088)
Loss: 3.284 | Acc: 16.580% (191/1152)
Loss: 3.230 | Acc: 16.612% (202/1216)
Loss: 3.186 | Acc: 16.641% (213/1280)
Loss: 3.145 | Acc: 16.592% (223/1344)
Loss: 3.116 | Acc: 16.690% (235/1408)
Loss: 3.079 | Acc: 16.644% (245/1472)
Loss: 3.041 | Acc: 16.797% (258/1536)
Loss: 3.007 | Acc: 17.000% (272/1600)
Loss: 2.978 | Acc: 17.188% (286/1664)
Loss: 2.942 | Acc: 17.419% (

Loss: 2.004 | Acc: 30.262% (4106/13568)
Loss: 2.004 | Acc: 30.238% (4122/13632)
Loss: 2.002 | Acc: 30.250% (4143/13696)
Loss: 2.001 | Acc: 30.269% (4165/13760)
Loss: 2.001 | Acc: 30.302% (4189/13824)
Loss: 1.999 | Acc: 30.343% (4214/13888)
Loss: 1.997 | Acc: 30.368% (4237/13952)
Loss: 1.996 | Acc: 30.387% (4259/14016)
Loss: 1.994 | Acc: 30.433% (4285/14080)
Loss: 1.993 | Acc: 30.444% (4306/14144)
Loss: 1.992 | Acc: 30.469% (4329/14208)
Loss: 1.990 | Acc: 30.521% (4356/14272)
Loss: 1.989 | Acc: 30.552% (4380/14336)
Loss: 1.987 | Acc: 30.618% (4409/14400)
Loss: 1.984 | Acc: 30.704% (4441/14464)
Loss: 1.983 | Acc: 30.775% (4471/14528)
Loss: 1.981 | Acc: 30.853% (4502/14592)
Loss: 1.979 | Acc: 30.888% (4527/14656)
Loss: 1.979 | Acc: 30.897% (4548/14720)
Loss: 1.977 | Acc: 30.979% (4580/14784)
Loss: 1.975 | Acc: 31.041% (4609/14848)
Loss: 1.974 | Acc: 31.029% (4627/14912)
Loss: 1.973 | Acc: 31.076% (4654/14976)
Loss: 1.970 | Acc: 31.203% (4693/15040)
Loss: 1.969 | Acc: 31.230% (4717/15104)


Loss: 1.819 | Acc: 35.624% (9530/26752)
Loss: 1.818 | Acc: 35.643% (9558/26816)
Loss: 1.817 | Acc: 35.636% (9579/26880)
Loss: 1.817 | Acc: 35.641% (9603/26944)
Loss: 1.816 | Acc: 35.682% (9637/27008)
Loss: 1.816 | Acc: 35.694% (9663/27072)
Loss: 1.815 | Acc: 35.724% (9694/27136)
Loss: 1.814 | Acc: 35.746% (9723/27200)
Loss: 1.813 | Acc: 35.780% (9755/27264)
Loss: 1.812 | Acc: 35.817% (9788/27328)
Loss: 1.812 | Acc: 35.832% (9815/27392)
Loss: 1.811 | Acc: 35.857% (9845/27456)
Loss: 1.810 | Acc: 35.883% (9875/27520)
Loss: 1.809 | Acc: 35.927% (9910/27584)
Loss: 1.808 | Acc: 35.952% (9940/27648)
Loss: 1.808 | Acc: 35.974% (9969/27712)
Loss: 1.807 | Acc: 35.992% (9997/27776)
Loss: 1.806 | Acc: 36.027% (10030/27840)
Loss: 1.805 | Acc: 36.070% (10065/27904)
Loss: 1.805 | Acc: 36.073% (10089/27968)
Loss: 1.804 | Acc: 36.077% (10113/28032)
Loss: 1.804 | Acc: 36.094% (10141/28096)
Loss: 1.804 | Acc: 36.094% (10164/28160)
Loss: 1.803 | Acc: 36.115% (10193/28224)
Loss: 1.803 | Acc: 36.139% (10223

Loss: 1.726 | Acc: 38.616% (15323/39680)
Loss: 1.725 | Acc: 38.632% (15354/39744)
Loss: 1.726 | Acc: 38.630% (15378/39808)
Loss: 1.725 | Acc: 38.654% (15412/39872)
Loss: 1.725 | Acc: 38.664% (15441/39936)
Loss: 1.724 | Acc: 38.688% (15475/40000)
Loss: 1.724 | Acc: 38.696% (15503/40064)
Loss: 1.723 | Acc: 38.719% (15537/40128)
Loss: 1.723 | Acc: 38.717% (15561/40192)
Loss: 1.722 | Acc: 38.735% (15593/40256)
Loss: 1.722 | Acc: 38.725% (15614/40320)
Loss: 1.722 | Acc: 38.760% (15653/40384)
Loss: 1.721 | Acc: 38.771% (15682/40448)
Loss: 1.721 | Acc: 38.789% (15714/40512)
Loss: 1.720 | Acc: 38.811% (15748/40576)
Loss: 1.720 | Acc: 38.826% (15779/40640)
Loss: 1.719 | Acc: 38.841% (15810/40704)
Loss: 1.719 | Acc: 38.859% (15842/40768)
Loss: 1.719 | Acc: 38.871% (15872/40832)
Loss: 1.719 | Acc: 38.882% (15901/40896)
Loss: 1.718 | Acc: 38.921% (15942/40960)
Loss: 1.717 | Acc: 38.931% (15971/41024)
Loss: 1.717 | Acc: 38.948% (16003/41088)
Loss: 1.717 | Acc: 38.963% (16034/41152)
Loss: 1.716 | Ac

Loss: 1.438 | Acc: 48.794% (1780/3648)
Loss: 1.437 | Acc: 48.680% (1807/3712)
Loss: 1.437 | Acc: 48.570% (1834/3776)
Loss: 1.433 | Acc: 48.724% (1871/3840)
Loss: 1.433 | Acc: 48.668% (1900/3904)
Loss: 1.431 | Acc: 48.715% (1933/3968)
Loss: 1.431 | Acc: 48.735% (1965/4032)
Loss: 1.434 | Acc: 48.657% (1993/4096)
Loss: 1.437 | Acc: 48.582% (2021/4160)
Loss: 1.436 | Acc: 48.698% (2057/4224)
Loss: 1.437 | Acc: 48.671% (2087/4288)
Loss: 1.436 | Acc: 48.782% (2123/4352)
Loss: 1.434 | Acc: 48.958% (2162/4416)
Loss: 1.430 | Acc: 49.107% (2200/4480)
Loss: 1.428 | Acc: 49.186% (2235/4544)
Loss: 1.429 | Acc: 49.089% (2262/4608)
Loss: 1.429 | Acc: 49.101% (2294/4672)
Loss: 1.428 | Acc: 49.050% (2323/4736)
Loss: 1.430 | Acc: 49.042% (2354/4800)
Loss: 1.429 | Acc: 49.075% (2387/4864)
Loss: 1.428 | Acc: 49.168% (2423/4928)
Loss: 1.428 | Acc: 49.159% (2454/4992)
Loss: 1.428 | Acc: 49.110% (2483/5056)
Loss: 1.430 | Acc: 48.984% (2508/5120)
Loss: 1.429 | Acc: 49.055% (2543/5184)
Loss: 1.431 | Acc: 49.009

...

Loss: 1.138 | Acc: 60.938% (39/64)
Loss: 1.161 | Acc: 60.938% (78/128)
Loss: 1.122 | Acc: 61.458% (118/192)
Loss: 1.151 | Acc: 58.984% (151/256)
Loss: 1.143 | Acc: 58.125% (186/320)
Loss: 1.169 | Acc: 57.292% (220/384)
Loss: 1.170 | Acc: 58.259% (261/448)
Loss: 1.195 | Acc: 58.008% (297/512)
Loss: 1.187 | Acc: 58.333% (336/576)
Loss: 1.191 | Acc: 58.125% (372/640)
Loss: 1.186 | Acc: 58.097% (409/704)
Loss: 1.194 | Acc: 57.943% (445/768)
Loss: 1.219 | Acc: 57.212% (476/832)
Loss: 1.236 | Acc: 56.808% (509/896)
Loss: 1.241 | Acc: 56.354% (541/960)
Loss: 1.247 | Acc: 56.055% (574/1024)
Loss: 1.242 | Acc: 56.250% (612/1088)
Loss: 1.242 | Acc: 56.337% (649/1152)
Loss: 1.238 | Acc: 56.497% (687/1216)
Loss: 1.225 | Acc: 57.109% (731/1280)
Loss: 1.220 | Acc: 56.994% (766/1344)
Loss: 1.221 | Acc: 57.031% (803/1408)
Loss: 1.222 | Acc: 56.522% (832/1472)
Loss: 1.229 | Acc: 56.120% (862/1536)
Loss: 1.233 | Acc: 55.812% (893/1600)
Loss: 1.231 | Acc: 55.950% (931/1664)
Loss: 1.232 | Acc: 55.671% (96

Loss: 1.272 | Acc: 54.393% (7380/13568)
Loss: 1.272 | Acc: 54.401% (7416/13632)
Loss: 1.272 | Acc: 54.381% (7448/13696)
Loss: 1.271 | Acc: 54.404% (7486/13760)
Loss: 1.271 | Acc: 54.405% (7521/13824)
Loss: 1.272 | Acc: 54.385% (7553/13888)
Loss: 1.272 | Acc: 54.379% (7587/13952)
Loss: 1.272 | Acc: 54.374% (7621/14016)
Loss: 1.272 | Acc: 54.389% (7658/14080)
Loss: 1.272 | Acc: 54.355% (7688/14144)
Loss: 1.273 | Acc: 54.307% (7716/14208)
Loss: 1.273 | Acc: 54.288% (7748/14272)
Loss: 1.274 | Acc: 54.283% (7782/14336)
Loss: 1.273 | Acc: 54.299% (7819/14400)
Loss: 1.273 | Acc: 54.252% (7847/14464)
Loss: 1.275 | Acc: 54.240% (7880/14528)
Loss: 1.274 | Acc: 54.249% (7916/14592)
Loss: 1.273 | Acc: 54.244% (7950/14656)
Loss: 1.274 | Acc: 54.192% (7977/14720)
Loss: 1.273 | Acc: 54.241% (8019/14784)
Loss: 1.273 | Acc: 54.277% (8059/14848)
Loss: 1.273 | Acc: 54.265% (8092/14912)
Loss: 1.273 | Acc: 54.240% (8123/14976)
Loss: 1.273 | Acc: 54.262% (8161/15040)
Loss: 1.274 | Acc: 54.264% (8196/15104)


Loss: 1.275 | Acc: 54.295% (14386/26496)
Loss: 1.274 | Acc: 54.300% (14422/26560)
Loss: 1.274 | Acc: 54.312% (14460/26624)
Loss: 1.274 | Acc: 54.320% (14497/26688)
Loss: 1.274 | Acc: 54.302% (14527/26752)
Loss: 1.274 | Acc: 54.303% (14562/26816)
Loss: 1.274 | Acc: 54.293% (14594/26880)
Loss: 1.274 | Acc: 54.279% (14625/26944)
Loss: 1.274 | Acc: 54.284% (14661/27008)
Loss: 1.274 | Acc: 54.263% (14690/27072)
Loss: 1.274 | Acc: 54.245% (14720/27136)
Loss: 1.275 | Acc: 54.228% (14750/27200)
Loss: 1.275 | Acc: 54.225% (14784/27264)
Loss: 1.275 | Acc: 54.194% (14810/27328)
Loss: 1.276 | Acc: 54.209% (14849/27392)
Loss: 1.276 | Acc: 54.185% (14877/27456)
Loss: 1.276 | Acc: 54.208% (14918/27520)
Loss: 1.277 | Acc: 54.194% (14949/27584)
Loss: 1.277 | Acc: 54.192% (14983/27648)
Loss: 1.277 | Acc: 54.193% (15018/27712)
Loss: 1.277 | Acc: 54.205% (15056/27776)
Loss: 1.277 | Acc: 54.174% (15082/27840)
Loss: 1.277 | Acc: 54.175% (15117/27904)
Loss: 1.277 | Acc: 54.191% (15156/27968)
Loss: 1.276 | Ac

Loss: 1.269 | Acc: 54.334% (21351/39296)
Loss: 1.269 | Acc: 54.324% (21382/39360)
Loss: 1.269 | Acc: 54.327% (21418/39424)
Loss: 1.269 | Acc: 54.343% (21459/39488)
Loss: 1.269 | Acc: 54.339% (21492/39552)
Loss: 1.269 | Acc: 54.324% (21521/39616)
Loss: 1.269 | Acc: 54.325% (21556/39680)
Loss: 1.269 | Acc: 54.338% (21596/39744)
Loss: 1.269 | Acc: 54.328% (21627/39808)
Loss: 1.269 | Acc: 54.316% (21657/39872)
Loss: 1.269 | Acc: 54.314% (21691/39936)
Loss: 1.269 | Acc: 54.310% (21724/40000)
Loss: 1.269 | Acc: 54.321% (21763/40064)
Loss: 1.269 | Acc: 54.321% (21798/40128)
Loss: 1.269 | Acc: 54.319% (21832/40192)
Loss: 1.269 | Acc: 54.322% (21868/40256)
Loss: 1.268 | Acc: 54.320% (21902/40320)
Loss: 1.268 | Acc: 54.321% (21937/40384)
Loss: 1.268 | Acc: 54.319% (21971/40448)
Loss: 1.268 | Acc: 54.325% (22008/40512)
Loss: 1.268 | Acc: 54.330% (22045/40576)
Loss: 1.268 | Acc: 54.328% (22079/40640)
Loss: 1.268 | Acc: 54.346% (22121/40704)
Loss: 1.268 | Acc: 54.344% (22155/40768)
Loss: 1.268 | Ac

Loss: 1.282 | Acc: 53.571% (1680/3136)
Loss: 1.280 | Acc: 53.594% (1715/3200)
Loss: 1.278 | Acc: 53.646% (1751/3264)
Loss: 1.277 | Acc: 53.786% (1790/3328)
Loss: 1.276 | Acc: 53.862% (1827/3392)
Loss: 1.278 | Acc: 53.819% (1860/3456)
Loss: 1.281 | Acc: 53.750% (1892/3520)
Loss: 1.281 | Acc: 53.739% (1926/3584)
Loss: 1.281 | Acc: 53.810% (1963/3648)
Loss: 1.280 | Acc: 53.798% (1997/3712)
Loss: 1.280 | Acc: 53.814% (2032/3776)
Loss: 1.277 | Acc: 54.010% (2074/3840)
Loss: 1.275 | Acc: 54.098% (2112/3904)
Loss: 1.277 | Acc: 54.057% (2145/3968)
Loss: 1.275 | Acc: 54.191% (2185/4032)
Loss: 1.275 | Acc: 54.175% (2219/4096)
Loss: 1.276 | Acc: 54.183% (2254/4160)
Loss: 1.275 | Acc: 54.190% (2289/4224)
Loss: 1.274 | Acc: 54.314% (2329/4288)
Loss: 1.273 | Acc: 54.389% (2367/4352)
Loss: 1.273 | Acc: 54.438% (2404/4416)
Loss: 1.272 | Acc: 54.531% (2443/4480)
Loss: 1.270 | Acc: 54.577% (2480/4544)
Loss: 1.272 | Acc: 54.514% (2512/4608)
Loss: 1.271 | Acc: 54.495% (2546/4672)
Loss: 1.267 | Acc: 54.540